In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## Embedding

임베딩 클래스는 텍스트 임베딩 모델과의 인터페이스를 위해 설계된 클래스입니다.

이 클래스는 다양한 임베딩 모델(OpenAI, Cohere, HuggingFace 등)에 대한 표준 인터페이스를 제공하도록 설계되었습니다.

임베딩은 텍스트 조각의 벡터 표현을 생성합니다. 이는 벡터 공간에서 텍스트를 생각하고, 벡터 공간에서 가장 유사한 텍스트 조각을 찾는 시맨틱 검색과 같은 작업을 수행할 수 있다는 점에서 유용합니다.

LangChain의 기본 Embeddings 클래스는 다음의 2가지 형태의 임베딩 함수를 가집니다.

- 문서 임베딩(`embed_documents`): 여러 개의 텍스트를 입력
- 쿼리 임베딩(`embed_query`): 하나의 텍스트를 입력


In [53]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.callbacks import get_openai_callback

openai_embedding = OpenAIEmbeddings()

with get_openai_callback() as cb:
    embedding = openai_embedding.embed_query(
        "안녕하세요? 반갑습니다. 제 이름은 테디입니다."
    )
    print(f"호출에 청구된 금액(USD): \t${cb.total_cost}")
len(embedding)

호출에 청구된 금액(USD): 	$0.0


1536

In [54]:
embedding = openai_embedding.embed_documents(
    ["안녕하세요?", "반갑습니다", "제 이름은 테디입니다."])
len(embedding), len(embedding[0])

(3, 1536)

In [69]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./data/appendix-keywords.txt")
docs = loader.load()
len(docs)

1

In [71]:
with get_openai_callback() as cb:
    openai_embedding.embed_query(docs[0].page_content)
    print(f"호출에 청구된 금액(USD): \t${cb.total_cost:.2f}")

호출에 청구된 금액(USD): 	$0.00


In [73]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
pdf = loader.load()
len(pdf)

23

In [79]:
whole_pdf = " ".join([p.page_content for p in pdf])
len(whole_pdf)

28226

In [80]:
with get_openai_callback() as cb:
    openai_embedding.embed_query(whole_pdf)
    print(f"호출에 청구된 금액(USD): \t${cb.total_cost:.2f}")

호출에 청구된 금액(USD): 	$0.00


## HuggingFace Embeddings


In [47]:
from langchain_community.embeddings import HuggingFaceEmbeddings

hf_embedding = HuggingFaceEmbeddings()

text = "안녕하세요? 반갑습니다. 제 이름은 테디입니다."
query_result = hf_embedding.embed_query(text)

In [48]:
len(query_result)

768

In [43]:
import os
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

# HugginFace 링크: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
hf_embedding = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    model_name="sentence-transformers/all-MiniLM-l6-v2",
)

query_result = hf_embedding.embed_query(text)
len(query_result)

384

허깅페이스의 `BGE` 모델은 최고의 오픈소스 임베딩 모델입니다. BGE 모델은 Beijing Academy of Artificial Intelligence (BAAI) 에서 개발했습니다. BAAI는 AI 연구 및 개발에 종사하는 민간 비영리 단체입니다.


In [45]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
bge_embedding = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [46]:
query_result = bge_embedding.embed_query(text)
len(query_result)

384

## CacheBackedEmbeddings


`CacheBackedEmbeddings` 는 임베딩을 저장하거나 임시로 캐싱(caching)하여 다시 계산할 필요가 없도록 할 수 있습니다.

`CacheBackedEmbeddings` 는 임베더를 감싸는 wrapper 클래스로, 임베딩을 key-value 저장소에 캐시합니다.

텍스트는 해시(hash) 되고 해시는 캐시에서 키(key)로 사용됩니다.

`CacheBackedEmbedding` 을 초기화하는 데 주로 지원되는 방법은 `from_bytes_store` 입니다. 여기에는 다음 매개변수가 사용됩니다:

- `underlying_embedder`: 임베딩에 사용할 임베더.
- `document_embedding_cache`: 문서 임베딩을 캐싱하기 위한 바이트 저장소.
- `namespace`: (선택 사항, 기본값은 "") 문서 캐시에 사용할 네임스페이스입니다. 이 네임스페이스는 다른 캐시와의 충돌을 피하기 위해 사용됩니다. 예를 들어 사용된 임베딩 모델의 이름으로 설정합니다.

주의: 서로 다른 임베딩 모델을 사용하여 임베드된 동일한 텍스트의 충돌을 방지하려면 네임스페이스 매개변수를 설정해야 합니다.


In [11]:
from langchain.embeddings import CacheBackedEmbeddings

In [12]:
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

openai_embedding = OpenAIEmbeddings()

store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    openai_embedding, store, namespace=openai_embedding.model
)

In [14]:
# 초기의 캐시는 비어있음
list(store.yield_keys())

[]

In [17]:
cached_embedder.embed_query("안녕하세요? 반갑습니다. 제 이름은 테디입니다.")

[-0.010785634520275275,
 -0.02857525804032248,
 -0.013868152954896452,
 -0.029109138700817806,
 -0.0014324174117619306,
 0.037117328119151284,
 -0.019728114628814294,
 0.0016810844947886372,
 -0.007029411777495212,
 0.01148476253748001,
 0.0020973833531222792,
 -0.011904239534067364,
 -0.010950882808307248,
 -0.013296139291969409,
 0.003098407263244605,
 -0.006412908369967746,
 0.013156313315999436,
 -0.009272976684602963,
 0.011948728813006503,
 -0.0177451333097855,
 0.005049609062381412,
 -0.01752904039762953,
 0.0003384413978292806,
 0.0016302388229267532,
 5.392420225686565e-05,
 0.0007694378068101522,
 0.035795341814895514,
 -0.011065285540892656,
 0.012012285990145349,
 -0.02608382116682805,
 0.018812892768131025,
 0.0052053238669748135,
 -0.005964830923064681,
 -0.009539916083528062,
 -0.002993538014097767,
 -0.016105361119835498,
 0.01174534659122025,
 -0.003289078328999859,
 0.01801207457135573,
 0.005612089241869885,
 -0.00309999609954082,
 -0.015546059078600736,
 0.001021679

## ByteStore 로 교체


다른 바이트 저장소를 사용하려면 `CacheBackedEmbedding` 을 생성할 때 사용하면 됩니다.

아래에서는 동등한 캐시된 임베딩 객체를 생성하지만, 비영구적인 `InMemoryByteStore` 를 대신 사용합니다:


In [1]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore

store = InMemoryByteStore()

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)

NameError: name 'underlying_embeddings' is not defined